In [16]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler

from autoencoder import Autoencoder

In [ ]:
# Load model

model_path = 'output/experiment_MSELoss_SGD_lr_0.001_5/model_399.pth'

model = Autoencoder(train.values.shape[1])
model.load_state_dict(torch.load(model_path))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

TypeError: Autoencoder.__init__() missing 1 required positional argument: 'input_size'

In [9]:
# Load training and validation data from datasets
# Files data/care_to_compare/Wind Farm A/datasets/25
# Files data/care_to_compare/Wind Farm A/datasets/69
# Files data/care_to_compare/Wind Farm A/datasets/13

# Load csv file
df_25 = pd.read_csv('../../data/care_to_compare/Wind Farm A/datasets/25.csv', delimiter=';')
df_69 = pd.read_csv('../../data/care_to_compare/Wind Farm A/datasets/69.csv', delimiter=';')
df_13 = pd.read_csv('../../data/care_to_compare/Wind Farm A/datasets/13.csv', delimiter=';')

# Combine dataframes
#df = pd.concat([df_25, df_69, df_13])
df = df_25
df.head()

,time_stamp,asset_id,id,train_test,status_type_id,sensor_0_avg,sensor_1_avg,sensor_2_avg,wind_speed_3_avg,wind_speed_4_avg,...,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,sensor_52_avg,sensor_52_max,sensor_52_min,sensor_52_std,sensor_53_avg
0,2022-05-22 06:50:00,11,0,train,5,18.0,161.3,156.2,2.4,2.4,...,-130.0,0.0,0.0,-682.0,-130.0,0.0,0.0,0.0,0.0,19.0
1,2022-05-22 07:00:00,11,1,train,5,18.0,172.3,167.1,2.8,2.8,...,-972.0,0.0,0.0,-996.0,-972.0,0.0,0.0,0.0,0.0,19.0
2,2022-05-22 07:10:00,11,2,train,5,18.0,173.0,41.0,3.1,3.1,...,-542.0,0.0,0.0,-849.0,-542.0,2.0,2.2,0.0,0.3,20.0
3,2022-05-22 07:20:00,11,3,train,5,18.0,168.4,5.6,2.4,2.4,...,-327.0,0.0,0.0,-781.0,-327.0,1.5,2.1,0.0,0.5,20.0
4,2022-05-22 07:30:00,11,4,train,5,18.0,155.4,-18.6,2.3,2.3,...,-253.0,0.0,0.0,-748.0,-253.0,1.2,1.9,0.0,0.8,20.0


In [10]:
# Filter out all train data (all rows where 'train_terst' column is 'train')
train_data = df[df['train_test'] == 'train']
test_data = df[df['train_test'] == 'prediction']

# Remove the first 5 columns (not needed for training)
train = train_data.iloc[:, 5:]
test = test_data.iloc[:, 5:]

# Remove column that contains 'sensor_1', 'sensor_2', 'sensor_5', 'sensor_42'
train = train.drop(columns=['sensor_0_avg', 'sensor_1_avg', 'sensor_2_avg', 'wind_speed_4_avg', 'sensor_5_avg', 'sensor_5_max', 'sensor_5_min', 'sensor_42_avg'])
test = test.drop(columns=['sensor_0_avg', 'sensor_1_avg', 'sensor_2_avg', 'wind_speed_4_avg', 'sensor_5_avg', 'sensor_5_max', 'sensor_5_min', 'sensor_42_avg'])

#scaler = StandardScaler()
# data_normalized = scaler.fit_transform(train.values)

scaler = MinMaxScaler()
train_data_normalized = scaler.fit_transform(train.values)
test_data_normalized = scaler.fit_transform(test.values)

# Create tensors  
train_data_normalized = torch.tensor(train_data_normalized, dtype=torch.float32)
test_data_normalized = torch.tensor(test_data_normalized, dtype=torch.float32)

# Create a DataLoader
batch_size = 128
dataset = TensorDataset(train_data_normalized)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [11]:
# Test the model on the test data
# From: https://github.com/curiousily/Getting-Things-Done-with-Pytorch/blob/master/manuscript/06.time-series-anomaly-ecg.md

def predict(model, dataset):
  predictions, losses = [], []
  criterion = torch.nn.L1Loss(reduction='sum').to(device)
  with torch.no_grad():
    model = model.eval()
    for seq_true in dataset:
      seq_true = seq_true.to(device)
      seq_pred = model(seq_true)

      loss = criterion(seq_pred, seq_true)

      predictions.append(seq_pred.cpu().numpy().flatten())
      losses.append(loss.item())
  return predictions, losses

_, losses = predict(model, train_data_normalized)

sns.displot(losses, bins=50, kde=True);

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [ ]:
THRESHOLD = 13

predictions, pred_losses = predict(model, test_data_normalized)
sns.displot(pred_losses, bins=50, kde=True);

correct = sum(l <= THRESHOLD for l in pred_losses)
print(f'Correct normal predictions: {correct}/{len(test_data_normalized)}')

In [ ]:
# Anamoly dataset
df_68 = pd.read_csv('../../data/care_to_compare/Wind Farm A/datasets/68.csv', delimiter=';')

anamoly_data = df_68.iloc[:, 5:]
anamoly_data = anamoly_data.drop(columns=['sensor_0_avg', 'sensor_1_avg', 'sensor_2_avg', 'wind_speed_4_avg', 'sensor_5_avg', 'sensor_5_max', 'sensor_5_min', 'sensor_42_avg'])

scaler = MinMaxScaler()
anamoly_data_normalized = scaler.fit_transform(anamoly_data.values)
anamoly_data_normalized = torch.tensor(anamoly_data_normalized, dtype=torch.float32)

In [ ]:
predictions, pred_losses = predict(model, anamoly_data_normalized)

# Plot a time series of the prediction losses
plt.figure(figsize=(10, 6))
plt.plot(pred_losses, label='Loss')
plt.axhline(THRESHOLD, color='r', label='Threshold')
plt.xlabel('Time step')
plt.ylabel('Loss')
plt.legend()

sns.displot(pred_losses, bins=50, kde=True);

correct = sum(l > THRESHOLD for l in pred_losses)
print(f'Correct anomaly predictions: {correct}/{len(anamoly_data_normalized)}')